In [1]:
import torch
from torch import nn
import numpy as np
from tqdm import tqdm
import pandas as pd
import os
import json
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
import torch.nn.functional as F

In [2]:
with open('fin_word.json', 'r', encoding='utf-8') as f:
    word2idx = json.load(f)
with open('fin_tag.json', 'r', encoding='utf-8') as f:
    tag2idx = json.load(f)

In [3]:
word2idx['<pad>'] = 0
idx2word = {word2idx[w]:w for w in word2idx}
idx2tag = {tag2idx[t]:t for t in tag2idx}

In [4]:
word_limit = 40 ## 每句話的最長長度
sentence_limit = 80 ## 每篇判決的句子數目

In [5]:
from torch.utils.data import TensorDataset, DataLoader ,SubsetRandomSampler ,ConcatDataset ,Dataset
class HANDataset(Dataset):
    def __init__(self,word_pad_idx,num_classes , word_limit = 40 ,sentence_limit = 80 ):
        # Load data
        self.data = torch.load('FIN_data.pth.tar')
        self.word_limit = word_limit
        self.sentence_limit = sentence_limit
        self.word_pad_idx = word_pad_idx
        self.num_classes = num_classes

    def __getitem__(self, id_doc):
        return (self.data['docs'][id_doc], \
               self.data['sentences_per_document'][id_doc], \
               self.data['words_per_sentence'][id_doc], \
               self.data['labels'][id_doc])

    def __len__(self):
        return len(self.data['labels'])
    
    def one_hot(self,tag):
        bs_oh_label = torch.tensor([0.0] * self.num_classes)
        for t in tag:
            bs_oh_label += torch.eye(self.num_classes)[t]
        return bs_oh_label.tolist()
    
    def turncut_dlen(self,x):
        if len(x) > self.max_doc_len:
            return x[:self.max_doc_len]
        elif len(x) < self.max_doc_len:
            return x + [0] * (self.max_doc_len-len(x))
        else:
            return x
        
    def max_slen(self,x):
        if x > self.max_sent_len:
            return self.max_sent_len
        else:
            return x
        
    def max_dlen(self,x):
        if x > self.max_doc_len:
            return self.max_doc_len
        else:
            return x
        
    def collate_fn(self, col_datasets):
        bs_doc ,bs_dlen,bs_slen,bs_label = [],[],[],[]
        for c_data in col_datasets:
            bs_doc.append(c_data[0])
            bs_dlen.append(c_data[1])
            bs_slen.append(c_data[2])
            bs_label.append(c_data[3])
        self.max_sent_len = min( max([lens for slen in bs_slen for lens in slen ]) ,self.word_limit )
        self.max_doc_len = min( self.sentence_limit , max(bs_dlen))
        pad_docs = []
        pad_labels = []
        for doc,label in zip(bs_doc,bs_label):
            pad_doc = []
            for sent in doc:
                if len(sent) > self.max_sent_len:
                    pad_doc.append(sent[:self.max_sent_len])
                else:
                    pad_doc.append(sent + (self.max_sent_len-len(sent))*[self.word_pad_idx])
            if len(pad_doc) > self.max_doc_len:
                pad_doc = pad_doc[:self.max_doc_len]
            else:
                pad_doc.extend((self.max_doc_len-len(pad_doc)) * [[self.word_pad_idx] * self.max_sent_len])
            pad_docs.append(pad_doc)
            pad_labels.append(self.one_hot(label))
        bs_slen_p = []
        for b_s in list(map(self.turncut_dlen ,bs_slen)):
            bs_slen_p.append(list(map(self.max_slen ,b_s)))
        bs_dlen2 = list(map(self.max_dlen,bs_dlen))
        return torch.LongTensor(pad_docs) , torch.LongTensor(bs_dlen2), torch.LongTensor(bs_slen_p), torch.tensor(pad_labels)     
        

In [6]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, PackedSequence


class HierarchialAttentionNetwork(nn.Module):
    def __init__(self, n_classes, vocab_size, emb_size, word_rnn_size, sentence_rnn_size, word_rnn_layers,
                 sentence_rnn_layers, word_att_size, sentence_att_size, dropout=0.5):
        super(HierarchialAttentionNetwork, self).__init__()
        self.sentence_attention = SentenceAttention(vocab_size, emb_size, word_rnn_size, sentence_rnn_size,
                                                    word_rnn_layers, sentence_rnn_layers, word_att_size,
                                                    sentence_att_size, dropout)

        self.fc = nn.Linear(2 * sentence_rnn_size, n_classes)
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        
        self.dropout = nn.Dropout(dropout)

    def forward(self, documents, sentences_per_document, words_per_sentence):
        document_embeddings, word_alphas, sentence_alphas = self.sentence_attention(documents, sentences_per_document,
                                                                                    words_per_sentence)  
        scores = self.sigmoid(self.fc(self.dropout(document_embeddings)) ) 
        
        return scores, word_alphas, sentence_alphas


class SentenceAttention(nn.Module):
    def __init__(self, vocab_size, emb_size, word_rnn_size, sentence_rnn_size, word_rnn_layers, sentence_rnn_layers,
                 word_att_size, sentence_att_size, dropout):
        super(SentenceAttention, self).__init__()

        self.word_attention = WordAttention(vocab_size, emb_size, word_rnn_size, word_rnn_layers, word_att_size,
                                            dropout)

        self.sentence_rnn = nn.GRU(2 * word_rnn_size, sentence_rnn_size, num_layers=sentence_rnn_layers,
                                   bidirectional=True, dropout=dropout, batch_first=True)

        self.sentence_attention = nn.Linear(2 * sentence_rnn_size, sentence_att_size)

        self.sentence_context_vector = nn.Linear(sentence_att_size, 1,
                                                 bias=False) 

        self.dropout = nn.Dropout(dropout)

    def forward(self, documents, sentences_per_document, words_per_sentence):
        packed_sentences = pack_padded_sequence(documents,
                                                lengths=sentences_per_document.tolist(),
                                                batch_first=True,
                                                enforce_sorted=False) 
        packed_words_per_sentence = pack_padded_sequence(words_per_sentence,
                                                         lengths=sentences_per_document.tolist(),
                                                         batch_first=True,
                                                         enforce_sorted=False)  
        sentences, word_alphas = self.word_attention(packed_sentences.data,
                                                     packed_words_per_sentence.data)  
        
        sentences = self.dropout(sentences)
        
        packed_sentences, _ = self.sentence_rnn(PackedSequence(data=sentences,
                                                               batch_sizes=packed_sentences.batch_sizes,
                                                               sorted_indices=packed_sentences.sorted_indices,
                                                               unsorted_indices=packed_sentences.unsorted_indices))  
        att_s = self.sentence_attention(packed_sentences.data)  
        att_s = torch.tanh(att_s)  
        
        att_s = self.sentence_context_vector(att_s).squeeze(1)  

        max_value = att_s.max()  
        att_s = torch.exp(att_s - max_value) 

        att_s, _ = pad_packed_sequence(PackedSequence(data=att_s,
                                                      batch_sizes=packed_sentences.batch_sizes,
                                                      sorted_indices=packed_sentences.sorted_indices,
                                                      unsorted_indices=packed_sentences.unsorted_indices),
                                       batch_first=True)  # (n_documents, max(sentences_per_document))

  
        sentence_alphas = att_s / torch.sum(att_s, dim=1, keepdim=True)

        documents, _ = pad_packed_sequence(packed_sentences,
                                           batch_first=True)  
        documents = documents * sentence_alphas.unsqueeze(2)  
        documents = documents.sum(dim=1)
        word_alphas, _ = pad_packed_sequence(PackedSequence(data=word_alphas,
                                                            batch_sizes=packed_sentences.batch_sizes,
                                                            sorted_indices=packed_sentences.sorted_indices,
                                                            unsorted_indices=packed_sentences.unsorted_indices),
                                             batch_first=True) 
        return documents, word_alphas, sentence_alphas


class WordAttention(nn.Module):
    def __init__(self, vocab_size, emb_size, word_rnn_size, word_rnn_layers, word_att_size, dropout):
        super(WordAttention, self).__init__()

        self.embeddings = nn.Embedding(vocab_size, emb_size)
        self.word_rnn = nn.GRU(emb_size, word_rnn_size, num_layers=word_rnn_layers, bidirectional=True,
                               dropout=dropout, batch_first=True)

        self.word_attention = nn.Linear(2 * word_rnn_size, word_att_size)
        self.word_context_vector = nn.Linear(word_att_size, 1, bias=False)

        self.dropout = nn.Dropout(dropout)

    def init_embeddings(self, embeddings):
        self.embeddings.weight = nn.Parameter(embeddings)

    def fine_tune_embeddings(self, fine_tune=False):
        for p in self.embeddings.parameters():
            p.requires_grad = fine_tune

    def forward(self, sentences, words_per_sentence):
        sentences = self.dropout(self.embeddings(sentences))
        packed_words = pack_padded_sequence(sentences,
                                            lengths=words_per_sentence.tolist(),
                                            batch_first=True,
                                            enforce_sorted=False) 
        # a PackedSequence object, where 'data' is the flattened words (n_words, word_emb)

        packed_words, _ = self.word_rnn(packed_words)  
        att_w = self.word_attention(packed_words.data)
        att_w = torch.tanh(att_w) 
        att_w = self.word_context_vector(att_w).squeeze(1)  # (n_words)

        max_value = att_w.max()  
        att_w = torch.exp(att_w - max_value)  
        
        att_w, _ = pad_packed_sequence(PackedSequence(data=att_w,batch_sizes=packed_words.batch_sizes,
                                    sorted_indices=packed_words.sorted_indices,
                                    unsorted_indices=packed_words.unsorted_indices),
                                       batch_first=True)
        word_alphas = att_w / torch.sum(att_w, dim=1, keepdim=True) 

        sentences, _ = pad_packed_sequence(packed_words,
                                           batch_first=True)  

        sentences = sentences * word_alphas.unsqueeze(2) 
        sentences = sentences.sum(dim=1) 

        return sentences, word_alphas

In [7]:
n_classes = len(tag2idx) 
word_rnn_size = 100  ## 超參數設定 可以調整這裡的參數
sentence_rnn_size = 100 
word_rnn_layers = 1  
sentence_rnn_layers = 1  
word_att_size = 200
sentence_att_size = 200  
dropout = 0

BS = 16
lr = 1e-4
epochs = 50

## 到這裡之前

k_folds = 10
kfold = KFold(n_splits=k_folds, shuffle=True)

PAD_IDX = word2idx['<pad>']
datasets = HANDataset(PAD_IDX,len(tag2idx))
dataloader = DataLoader(datasets , batch_size=BS, collate_fn =  datasets.collate_fn)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
# for i, (documents, sentences_per_document, words_per_sentence, labels) in enumerate(dataloader):
#     if i == 70:
#         print(documents)
#         print(sentences_per_document)
#         print(words_per_sentence)
#         print(labels)
#         print('')

In [9]:
from tqdm import trange
import torch.optim as optim
# class F1():
#     def __init__(self):
#         self.threshold = 0.5
#         self.n_precision = 0
#         self.n_recall = 0
#         self.n_corrects = 0
#         self.name = 'F1'

#     def reset(self):
#         self.n_precision = 0
#         self.n_recall = 0
#         self.n_corrects = 0

#     def update(self, predicts, groundTruth):
#         predicts = (predicts > self.threshold).float()
#         self.n_precision += torch.sum(predicts).data.item()
#         self.n_recall += torch.sum(groundTruth).data.item()
#         self.n_corrects += torch.sum(groundTruth * predicts).data.item()

#     def get_score(self):
#         recall = self.n_corrects / self.n_recall
#         precision = self.n_corrects / (self.n_precision + 1e-20) #prevent divided by zero
#         return 2 * (recall * precision) / (recall + precision + 1e-20)

#     def print_score(self):
#         score = self.get_score()
#         return '{:.5f}'.format(score)
    
def threshold_sc(x):
    if x > 0.5:
        return 1
    else:
        return 0

In [10]:
for fold, (train_ids, valid_ids) in enumerate(kfold.split(datasets)):
#     print(f'FOLD {fold}')
    print('--------------------------------')
#     print(valid_ids)
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    valid_subsampler = torch.utils.data.SubsetRandomSampler(valid_ids)
    train_dataloader = DataLoader(datasets, batch_size=BS,
                                  collate_fn=datasets.collate_fn,
                                  sampler=train_subsampler)
    valid_dataloader = DataLoader(datasets, batch_size=BS,
                                  collate_fn=datasets.collate_fn,
                                  sampler=valid_subsampler)
    model = HierarchialAttentionNetwork(n_classes=n_classes,
                                        vocab_size=len(word2idx),
                                        emb_size=300,
                                        word_rnn_size=word_rnn_size,
                                        sentence_rnn_size=sentence_rnn_size,
                                        word_rnn_layers=word_rnn_layers,
                                        sentence_rnn_layers=sentence_rnn_layers,
                                        word_att_size=word_att_size,
                                        sentence_att_size=sentence_att_size,
                                        dropout=dropout)
    model.sentence_attention.word_attention.fine_tune_embeddings(True)
    optimizer = optim.AdamW(params=filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
    criterion = nn.BCELoss()
#     criterion = BCEFocalLosswithLogits() ## 可以避免計算過多背景部分(答案為0) 
    model = model.to(device)
    criterion = criterion.to(device)
    all_loader = {
        'train' : train_dataloader,
        'valid' : valid_dataloader,
    }
    for epoch in trange(epochs):
        for loader in all_loader:
            loss = 0
            accuracy = []
            for i, (documents, sentences_per_document, words_per_sentence, labels) in enumerate(all_loader[loader]):
                documents = documents.to(device)
                sentences_per_document = sentences_per_document.to(device)  
                words_per_sentence = words_per_sentence.to(device)
                labels = labels.to(device)
                scores, word_alphas, sentence_alphas = model(documents, sentences_per_document,
                                                             words_per_sentence)
                bs_loss = criterion(scores ,labels)
                bs_loss.backward()
                optimizer.step()
                loss += bs_loss.item()
                optimizer.zero_grad()
                y_true = [list(map(lambda s:int(s),label)) for label in labels.cpu().tolist()]
                y_pred = [list(map(threshold_sc,sc)) for sc in scores.cpu().tolist()]
                bs_acc = f1_score(y_true,y_pred,average = 'micro')
                accuracy.append(bs_acc)
            print(f'{loader} accuracy:',np.mean(np.array(accuracy)), 'loss:',loss/ len(all_loader[loader]))
    break

--------------------------------


  0%|          | 0/50 [00:00<?, ?it/s]

train accuracy: 0.002756409024063779 loss: 0.18030077939455932


  2%|▏         | 1/50 [00:16<13:51, 16.97s/it]

valid accuracy: 0.0 loss: 0.041356902972266484
train accuracy: 0.0 loss: 0.03654604209950557


...

valid accuracy: 0.5782275661587425 loss: 0.014372530720523885
train accuracy: 0.5929708105938364 loss: 0.013797701360215922


 66%|██████▌   | 33/50 [08:02<04:11, 14.78s/it]

valid accuracy: 0.5843004325774908 loss: 0.014187120530452277


 66%|██████▌   | 33/50 [08:02<04:08, 14.63s/it]


KeyboardInterrupt: 

In [ ]:
valid_ids

array([  31,   39,   55,   65,   68,   70,  114,  121,  124,  129,  137,
        138,  143,  166,  175,  187,  188,  195,  199,  207,  211,  225,
        226,  230,  231,  237,  246,  250,  252,  256,  257,  275,  300,
        305,  312,  313,  320,  326,  331,  380,  381,  386,  392,  395,
        435,  437,  442,  446,  471,  476,  492,  498,  510,  527,  532,
        548,  551,  554,  557,  564,  591,  622,  630,  632,  638,  648,
        666,  668,  688,  691,  700,  701,  702,  720,  728,  747,  767,
        786,  792,  798,  800,  816,  820,  834,  843,  851,  857,  870,
        880,  889,  912,  917,  973,  974,  976,  984,  995, 1003, 1015,
       1037, 1050, 1058, 1074, 1080, 1086, 1096, 1105, 1108, 1145, 1151,
       1155, 1182, 1196, 1202, 1203, 1205, 1210, 1225, 1245, 1261, 1276,
       1287, 1292, 1293, 1314, 1315, 1318, 1323, 1325, 1347, 1361, 1362,
       1372, 1381, 1387, 1395, 1402, 1410, 1413, 1443, 1445, 1455, 1461,
       1465, 1467, 1469, 1480, 1499, 1519, 1532, 15

In [ ]:
idx = 1
test_subsampler = torch.utils.data.SubsetRandomSampler(np.array([idx]))
test_loader = DataLoader(datasets, batch_size=1,
                                  collate_fn=datasets.collate_fn,
                                  sampler=test_subsampler)

for test_data in test_loader:
    documents = test_data[0].to(device)
    print("Judgement : ")
    for sent in documents[0]:
        for word in sent:
            if word == 0:
                continue
            print(idx2word[word.cpu().item()],end = '')
        print()
    sentences_per_document = test_data[1].to(device)
    words_per_sentence = test_data[2].to(device)
    scores, word_alphas, sentence_alphas = model(documents, sentences_per_document,
                                                             words_per_sentence)
    print('\nPredict lawNames : ')
    for idx,s in enumerate(scores[0]):
        if s>0.5:
            print(idx2tag[idx])
#             p_law.append(idx)
    print('\nTrue lawNames : ')
    for idx,s in enumerate(test_data[3][0]):
        if s != 0:
            print(idx2tag[idx])

Judgement : 
福建金門地方法院刑事裁定104年度金訴字第1號上訴人即被告蔡銀英上列被告因違反銀行法案件
不服本院民國104年12月31日104年度金訴字第1號第一審刑事判決
提起上訴
本院裁定如下：主文上訴駁回
理由一、按上訴期間為10日
自送達判決後起算；原審法院認為上訴不合法律上之程式者
應以裁定駁回之
刑事訴訟法第349條前段
第362條前段定有明文
次按送達於在監獄或看守所之人
應囑託該監所長官為之
刑事訴訟法第56條第2項亦定有明文；又送達於應受送達人之住、居所、事務所或營業所
不獲會晤應受送達人
亦無法準用民事訴訟法第137條規定將文書付與有辨別事理能力之同居人或受僱人者
得將文書寄存送達地之自治或警察機關
並作送達通知書2份
1份黏貼於應受送達人住居所、事務所、營業所或其就業處所門首
另一份置於該送達處所信箱或其他適當位置
以為送達
又寄存送達
自寄存之日起
經10日發生效力
刑事訴訟法第62條準用民事訴訟法第136條至138條規定甚明；而民事訴訟法第138條第2項所稱之
係指
二、經查：本件上訴人即被告蔡銀英違反銀行法案件
其於本院審理時陳明其住所為金門縣○○鎮○○路00號
有本院歷次準備程序筆錄、審判筆錄附卷可稽
而本院於民國104年12月31日以104年度金訴字第1號刑事判決判處有期徒刑1年8月
緩刑3年
並應於判決確定1年內
向公庫支付新臺幣8萬元
犯罪所得新臺幣<unk>元沒收
該判決於105年1月6日寄存送達被告住所所在之金門縣警察局金湖分局金湖派出所
被告本人於105年1月7日15時14分許前往該警局領取等情
有個人戶籍資料查詢結果、本院送達證書、入出境資訊連結作業、金門縣警察局金湖分局金湖派出所受理司法文書寄存登記簿各1份在卷可查
又被告於斯時並未因案在監執行或羈押
此有臺灣高等法院在監在押全國紀錄表1份附卷可據
是被告並非在監獄或看守所之人
其主張該判決應囑託監所長官為送達云云
顯非可採
則依上開送達情形
該判決正本既已經被告於105年1月7日前往金湖派出所具領
是本件上訴期間應自105年1月8日起算10日
再依法院訴訟當事人在途期間標準第2條之規定
加計在途期間1日
是其上訴期間之末日應為105年1月18日屆滿
詎上訴人遲至107年3月13日始向本院提起上訴
有刑事上訴狀上本院收狀戳章為憑
顯已逾越法定上訴期間
且無從補